In [2]:
import bs4
import os
import pandas as pd

import confidential

from datetime import datetime, timedelta

def is_stream_span(el):
    target_class_list = 'TableCell__TableCellElement-sc-1nn7cfv-0 kJgiFu styled__RightTableCell-sc-135veyd-4 kGfYTK'.split(' ')
    if not el.has_attr('class'):
        return False
    return set(el['class']) == set(target_class_list) 

source_dir = "/home/aldakata/Projects/Tuebingen/Data Literacy/cultural-distance-from-spotify-data/spotify_data/"
source_file = "global_2023-01-05.html"

def extract_data(source_dir, source_file):
    """Function to extract data from the html files"""
    with open(f"{source_dir}/{source_file}", "r") as input_file:
        data = input_file.read()
    # Turn the content into soup
    soup = bs4.BeautifulSoup(data, features="html.parser")

    # Find all the rows in the html
    rows = soup.findAll("tr")
    df_rows = []

    for i, row in enumerate(rows[1:]):
        row_url = row.find('a', class_='Button-me270r-0', href=True)['href']
        row_data = {"spotify_url": row_url.replace('https://open.spotify.com/track/', '')}
        week_start = source_file.split(".")[0].split("_")[-1]
        row_data["week_start"] = week_start
        week_start.split("-")
        row_data["streams"] = int(row.find_all(is_stream_span)[0].get_text().replace(',', ''))
        row_data['country'] = source_file.split(".")[0].split("_")[0]
        row_data['ranking'] = i + 1
        df_rows.append(row_data)

    return pd.DataFrame(df_rows)

In [143]:
df = extract_data(source_dir, source_file)
df

,spotify_url,week_start,streams,country,ranking
0,https://open.spotify.com/track/1Qrg8KqiBpW07V7...,2023-01-05,41474366,global,1
1,https://open.spotify.com/track/3nqQXoyQOWXiESF...,2023-01-05,33132175,global,2
2,https://open.spotify.com/track/4Dvkj6JhhA12EX0...,2023-01-05,30998084,global,3
3,https://open.spotify.com/track/4uUG5RXrOk84mYE...,2023-01-05,30704256,global,4
4,https://open.spotify.com/track/2dHHgzDwk4BJdRw...,2023-01-05,28092232,global,5
...,...,...,...,...,...
195,https://open.spotify.com/track/4tYFy8ALRjIZvnv...,2023-01-05,6782777,global,196
196,https://open.spotify.com/track/37F7E7BKEw2E4O2...,2023-01-05,6775767,global,197
197,https://open.spotify.com/track/5g7sDjBhZ4I3gcF...,2023-01-05,6758678,global,198
198,https://open.spotify.com/track/3rb0tMq42Wfgguc...,2023-01-05,6748442,global,199


In [7]:
# list_df = []
htmls_to_parse = os.listdir(source_dir)
for i, file_path in enumerate(htmls_to_parse):
    if i>1236 and '.html' in file_path:
        print(f'[{i}/{len(htmls_to_parse)}] Parsing {file_path}')
        list_df.append(extract_data(source_dir, file_path))

[1237/1353] Parsing bo_2022-10-27.html
[1238/1353] Parsing cr_2022-12-08.html
[1239/1353] Parsing ar_2022-01-13.html
[1240/1353] Parsing bg_2022-06-30.html
[1241/1353] Parsing dk_2022-06-02.html
[1242/1353] Parsing cl_2022-06-02.html
[1243/1353] Parsing eg_2022-08-11.html
[1244/1353] Parsing ae_2022-09-01.html
[1245/1353] Parsing cz_2022-05-12.html
[1246/1353] Parsing ie_2022-06-09.html
[1247/1353] Parsing de_2022-12-15.html
[1248/1353] Parsing dk_2022-09-29.html
[1249/1353] Parsing br_2022-09-08.html
[1250/1353] Parsing cz_2022-03-31.html
[1251/1353] Parsing ch_2022-01-27.html
[1252/1353] Parsing ie_2022-08-04.html
[1253/1353] Parsing cr_2022-10-06.html
[1254/1353] Parsing ar_2022-04-28.html
[1255/1353] Parsing co_2022-03-24.html
[1256/1353] Parsing us_2022-12-08.html
[1257/1353] Parsing ch_2022-03-31.html
[1258/1353] Parsing au_2022-09-29.html
[1259/1353] Parsing ee_2022-05-12.html
[1260/1353] Parsing ie_2022-03-24.html
[1261/1353] Parsing es_2022-06-02.html
[1262/1353] Parsing ee_20

In [8]:
target_pickle = 'big_boy_df'
df = pd.concat(list_df)
df.to_pickle(f"{source_dir}/{target_pickle}.pkl")  

In [39]:
pd.__version__

'1.4.4'

In [14]:
source_pickle = 'big_boy_df'
df = pd.read_pickle(f'{source_dir}{source_pickle}.pkl')
df = df.reset_index()

In [22]:
len(set(df.spotify_url)), df

(10585,
         index             spotify_url  week_start  streams country  ranking
 0           0  2mnXxnrX5vCGolNkaFvVeM  2022-12-15   256566      do        1
 1           1  5Uq9MSXpOKkvxbk701GmYK  2022-12-15   171816      do        2
 2           2  1dnlWBC8oXaxzDQWtDxies  2022-12-15   164808      do        3
 3           3  6Sq7ltF9Qa7SNFBsV5Cogx  2022-12-15   148057      do        4
 4           4  5ww2BF9slyYgNOk37BlC4u  2022-12-15   134607      do        5
 ...       ...                     ...         ...      ...     ...      ...
 270394    195  6gBFPUFcJLzWGx4lenP6h2  2022-03-03   211304      ca      196
 270395    196  0I3q5fE6wg7LIfHGngUTnV  2022-03-03   210654      ca      197
 270396    197  3eJH2nAjvNXdmPfBkALiPZ  2022-03-03   210431      ca      198
 270397    198  7fBv7CLKzipRk6EC6TWHOB  2022-03-03   210308      ca      199
 270398    199  2KnLkZ3z7PO3kgVGHGqDpD  2022-03-03   210075      ca      200
 
 [270399 rows x 6 columns])

In [ ]:
tracks = []

In [36]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from time import sleep

cid = confidential.CID
secret = confidential.SECRET

client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret
)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_features(id):
    metadata = sp.track(id)
    features = sp.audio_features(id)

    # metadata
    name = metadata["name"]
    album = metadata["album"]["name"]
    artist = metadata["album"]["artists"][0]["name"]
    release_date = metadata["album"]["release_date"]
    length = metadata["duration_ms"]
    popularity = metadata["popularity"]

    # audio features
    acousticness = features[0]["acousticness"]
    danceability = features[0]["danceability"]
    energy = features[0]["energy"]
    instrumentalness = features[0]["instrumentalness"]
    liveness = features[0]["liveness"]
    loudness = features[0]["loudness"]
    speechiness = features[0]["speechiness"]
    tempo = features[0]["tempo"]
    time_signature = features[0]["time_signature"]
    url = id

    track = [
        name,
        album,
        artist,
        release_date,
        length,
        popularity,
        danceability,
        acousticness,
        energy,
        instrumentalness,
        liveness,
        loudness,
        speechiness,
        tempo,
        time_signature,
        url,
    ]
    return track



for i, url in enumerate(list(set(df.spotify_url))[3587:]):
    print(f'Processing... {i} {url}')
    track = get_track_features(url)
    tracks.append(track)
    if not i % 1000:
        print('Sleeping... xoxo')
        sleep(10)



df_api = pd.DataFrame(
    tracks,
    columns=[
        "name",
        "album",
        "artist",
        "release_date",
        "length",
        "popularity",
        "danceability",
        "acousticness",
        "energy",
        "instrumentalness",
        "liveness",
        "loudness",
        "speechiness",
        "tempo",
        "time_signature",
        "spotify_url",
    ],
)

Processing... 0 0vf2eBw2inhl8y61cYQMv2


KeyboardInterrupt: 

In [37]:
df_api = pd.DataFrame(
    tracks,
    columns=[
        "name",
        "album",
        "artist",
        "release_date",
        "length",
        "popularity",
        "danceability",
        "acousticness",
        "energy",
        "instrumentalness",
        "liveness",
        "loudness",
        "speechiness",
        "tempo",
        "time_signature",
        "spotify_url",
    ],
)

In [38]:
df_api.to_pickle('api_subset.pkl')

In [34]:
len(tracks)

3587

In [157]:
url = 'https://open.spotify.com/track/3nqQXoyQOWXiESF'
url.replace('https://open.spotify.com/track/', '')

'3nqQXoyQOWXiESF'

In [170]:
temp_urls = set([e.replace('https://open.spotify.com/track/', '') for e in df[df['week_start'].isin(['2022-12-15', '2022-08-25'])].spotify_url])
len(temp_urls)

1600

In [168]:
len(df[df['week_start'].isin(['2022-12-15', '2022-08-25'])])

1600

In [181]:
df_temp = pd.DataFrame(df[df['week_start'].isin(['2022-12-15', '2022-08-25'])][:1285])

In [210]:
df_api = pd.DataFrame(
    tracks,
    columns=[
        "name",
        "album",
        "artist",
        "release_date",
        "length",
        "popularity",
        "danceability",
        "acousticness",
        "energy",
        "instrumentalness",
        "liveness",
        "loudness",
        "speechiness",
        "tempo",
        "time_signature",
        "spotify_url",
    ],
)

In [213]:
df_api.spotify_url, df_temp.spotify_url

(0       0bYg9bo50gSsH3LtXe2SQn
 1       2FRnf9qhLbvw8fu4IBXx78
 2       1Qrg8KqiBpW07V7PNxwwwL
 3       2EjXfH91m7f8HiJN1yQg97
 4       3nqQXoyQOWXiESFLlDF1hG
                  ...          
 1280    6Y46tOTRhkBamosyuWa6YX
 1281    6wtZPYBIXUvCpXwVjMCJBf
 1282    2uky0VCI2EZni0nArOPznM
 1283    2JPLbjOn0wPCngEot2STUS
 1284    0ynhqjP6gIxoaGTWibGhjc
 Name: spotify_url, Length: 1285, dtype: object,
 0     0bYg9bo50gSsH3LtXe2SQn
 1     2FRnf9qhLbvw8fu4IBXx78
 2     1Qrg8KqiBpW07V7PNxwwwL
 3     2EjXfH91m7f8HiJN1yQg97
 4     3nqQXoyQOWXiESFLlDF1hG
                ...          
 80    6Y46tOTRhkBamosyuWa6YX
 81    6wtZPYBIXUvCpXwVjMCJBf
 82    2uky0VCI2EZni0nArOPznM
 83    2JPLbjOn0wPCngEot2STUS
 84    0ynhqjP6gIxoaGTWibGhjc
 Name: spotify_url, Length: 1285, dtype: object)

In [215]:
df_api.columns

Index(['name', 'album', 'artist', 'release_date', 'length', 'popularity',
       'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature',
       'spotify_url'],
      dtype='object')

In [216]:
df_temp.columns

Index(['spotify_url', 'week_start', 'streams', 'country', 'ranking'], dtype='object')

In [ ]:
pd.concat([df_api, df_temp])

In [233]:
df_joined  = df_api.set_index('spotify_url').join(df_temp.set_index('spotify_url') )
df_joined.head(30)

,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,week_start,streams,country,ranking
spotify_url,,,,,,,,,,,,,,,,,,,
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-12-15,80771,ie,97
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,120868,ie,24
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,6368731,global,156
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-12-15,80771,ie,97
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,120868,ie,24
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,6368731,global,156
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-12-15,80771,ie,97
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,120868,ie,24
003vvx7Niy0yvhvHt4a68B,Mr. Brightside,Hot Fuss,The Killers,2004,222973,87,0.352,0.00121,0.911,0.000000,0.0995,-5.230,0.0747,148.033,4,2022-08-25,6368731,global,156


In [234]:
len(df_joined), len(df_temp), len(df_api)

(2863, 1285, 1285)